In [ ]:
# import dependencies
from utils import *

In [ ]:
# set parameters (change manually if needed)
N_CPU = multiprocessing.cpu_count()-1
explicit_bias = False
datasets = ['ZINC', 'TOX21', 'PROTEINS']
models = ['GT', 'SAN', 'GraphiT']

if explicit_bias:
    states = ['train']
else:
    states = ['base', 'train']

In [ ]:
# generate plots
res = []
for dataset in datasets:
    for model in models:
        for state in states:
            if explicit_bias:
                key = (model, dataset, 'exp_bias', state)
            else:
                key = (model, dataset, state)
            print(key)
            try:
                tic = time.time()
                if explicit_bias:
                    spec = (model, dataset, 'sparse', 'NoPE', 'BN', 'ExplicitBias', 'Edge', ('Escore',), state, 0)
                else:
                    spec = (model, dataset, 'sparse', 'NoPE', 'BN', 'NoBias', 'Edge', ('Escore',), state, 0)
                a = load_spec(spec) # avoid loading multiple logs in parallel
                data_raw = get_values(a, use_abs=False, flatten=batch_median)
                del a
                # limit data to 50000 edges (proteins has more)
                params = [(data_raw[layer][:50000,:], key, layer) for layer in range(len(data_raw))]
                with multiprocessing.pool.ThreadPool(processes=min(N_CPU, len(data_raw))) as pool:
                    for item in pool.imap_unordered(ks_ll_plot_w, params):
                        res.append(item)
                del data_raw
                plt.close()
                toc = time.time()
                print(toc-tic)
            except Exception as e:
                print(e)